<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/parity_classification_final_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn matplotlib

In [ ]:
#!/usr/bin/env python3
"""
parity_classification_final.py

End-to-end 4-bit parity classification:

1) Exact analytic CNOT cascade (100% accuracy)
2) Variational QNN with 5-fold cross-validation
3) Simplified quantum-metric learning:
   • AngleEmbedding + baked-in parity CNOT layer
   • Single trainable Ry per qubit
   • Adam optimizer on centered kernel–target alignment
4) Final SVM with precomputed kernel and 5-fold CV
5) Heatmap of the learned kernel matrix

Dependencies:
    pip install pennylane numpy scikit-learn matplotlib
"""

import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# -----------------------------------------------------------------------------
# Hyperparameters
# -----------------------------------------------------------------------------
n_bits      = 4     # number of input qubits
qnn_epochs  = 30    # epochs for variational QNN
qnn_lr      = 0.4   # learning rate for QNN
n_steps     = 50    # training steps for metric learning
adam_lr     = 0.1   # learning rate for Adam optimizer
svm_C       = 1.0   # SVM regularization parameter
seed        = 42    # random seed

np.random.seed(seed)
pnp.random.seed(seed)

# -----------------------------------------------------------------------------
# 1) Generate 4-bit parity dataset
# -----------------------------------------------------------------------------
X = pnp.array(list(itertools.product([0, 1], repeat=n_bits)))
y = pnp.array(np.sum(X, axis=1) % 2)        # labels {0,1}
y_signed = 1 - 2 * y                        # labels {+1,-1} for regression

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

# -----------------------------------------------------------------------------
# 2) Analytic CNOT cascade
# -----------------------------------------------------------------------------
dev_exact = qml.device("default.qubit", wires=n_bits + 1)

@qml.qnode(dev_exact)
def analytic_circuit(x):
    # encode each bit
    for i, bit in enumerate(x):
        if bit:
            qml.PauliX(wires=i)
    # cascade CNOTs onto ancilla
    for i in range(n_bits):
        qml.CNOT(wires=[i, n_bits])
    # measure Z on ancilla: +1 even, -1 odd
    return qml.expval(qml.PauliZ(wires=n_bits))

preds_analytic = [0 if analytic_circuit(x) > 0 else 1 for x in X]
acc_analytic   = accuracy_score(y, preds_analytic)
print(f"Analytic CNOT Cascade Accuracy: {acc_analytic:.3f}")

# -----------------------------------------------------------------------------
# 3) Variational QNN w/ 5-fold CV
# -----------------------------------------------------------------------------
dev_qnn = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_qnn, interface="autograd")
def qnn_circuit(params, x):
    # angle-encode input
    for i, bit in enumerate(x):
        if bit:
            qml.RY(np.pi, wires=i)
    # variational layer
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    # nearest-neighbor entanglement
    for i in range(n_bits - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.expval(qml.PauliZ(wires=n_bits - 1))

var_scores = []
for train_idx, test_idx in kf.split(X, y):
    # initialize parameters
    params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
    opt    = qml.GradientDescentOptimizer(stepsize=qnn_lr)

    # training loop
    for _ in range(qnn_epochs):
        def cost(p):
            preds = [qnn_circuit(p, X[i]) for i in train_idx]
            return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
        params = opt.step(cost, params)

    # evaluation on held-out fold
    preds_test = [qnn_circuit(params, X[i]) for i in test_idx]
    bits       = [0 if v > 0 else 1 for v in preds_test]
    var_scores.append(accuracy_score(y[test_idx], bits))

print(f"Variational QNN CV Accuracy: {np.mean(var_scores):.3f} ± {np.std(var_scores):.3f}")

# -----------------------------------------------------------------------------
# 4) Simplified quantum-metric learning
# -----------------------------------------------------------------------------
dev_feat = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev_feat, interface="autograd")
def feature_map_shallow(params, x):
    # input embedding
    qml.templates.AngleEmbedding(x, wires=range(n_bits))
    # baked-in parity layer (CNOT ring)
    for i in range(n_bits):
        qml.CNOT(wires=[i, (i + 1) % n_bits])
    # single trainable Ry per qubit
    for i in range(n_bits):
        qml.RY(params[i], wires=i)
    return qml.state()

def build_kernel_matrix(params):
    # stack statevectors for all inputs
    states = pnp.stack([feature_map_shallow(params, x) for x in X])
    re, im = pnp.real(states), pnp.imag(states)
    # compute fidelity squared
    real_ov = re @ re.T + im @ im.T
    imag_ov = re @ im.T - im @ re.T
    return real_ov**2 + imag_ov**2

def centered_alignment(K):
    # center the Gram matrix
    N  = K.shape[0]
    H  = pnp.eye(N) - pnp.ones((N, N)) / N
    Kc = H @ K @ H
    # alignment with labels
    T  = pnp.outer(y_signed, y_signed)
    return pnp.sum(Kc * T) / (pnp.linalg.norm(Kc) * pnp.linalg.norm(T))

# initialize metric parameters
params_k = pnp.random.randn(n_bits, requires_grad=True) * 0.1
opt_metric = qml.AdamOptimizer(stepsize=adam_lr)

print("\nTraining feature-map via centered alignment:")
for step in range(n_steps):
    def loss(p):
        K = build_kernel_matrix(p)
        return -centered_alignment(K)
    params_k, cost = opt_metric.step_and_cost(loss, params_k)
    if step % 10 == 0:
        align = centered_alignment(build_kernel_matrix(params_k))
        print(f" Step {step:>3}: alignment = {align:.4f}")

# -----------------------------------------------------------------------------
# 5) Final SVM CV with learned kernel
# -----------------------------------------------------------------------------
K_final = np.array(build_kernel_matrix(params_k))
ker_scores = []

for tr, te in kf.split(K_final, y):
    svc = SVC(kernel="precomputed", C=svm_C)
    svc.fit(K_final[np.ix_(tr, tr)], y[tr])
    preds = svc.predict(K_final[np.ix_(te, tr)])
    ker_scores.append(accuracy_score(y[te], preds))

print(f"\nQuantum-Metric Kernel CV Accuracy: {np.mean(ker_scores):.3f} ± {np.std(ker_scores):.3f}")

# -----------------------------------------------------------------------------
# 6) Plot the learned kernel matrix
# -----------------------------------------------------------------------------
plt.figure(figsize=(5, 5))
plt.title("Learned Kernel Matrix (|⟨ψ_i|ψ_j⟩|²)")
plt.imshow(K_final, cmap="viridis", origin="lower")
plt.colorbar(label="Kernel value")
plt.xlabel("Index j")
plt.ylabel("Index i")
plt.tight_layout()
plt.show()